#### Section 1: Importing Necessary Libraries

In [41]:
### Basic Imports
import numpy as np
import pandas as pd
import random
import toml
import os
import logging
import math
import json
from datetime import datetime
import seaborn as sns
### Matplot Lib Imports
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

### Parallel Processing Libraries
from functools import partial
import time
from concurrent.futures import ProcessPoolExecutor, as_completed,ThreadPoolExecutor
from multiprocessing import Pool, cpu_count
from tqdm import tqdm
import concurrent.futures

### Scipy Imports
from scipy.spatial import distance
from shapely.geometry import Point, MultiPoint
from shapely.ops import cascaded_union
from scipy.spatial import distance
from sklearn.cluster import KMeans
import numpy as np
from scipy.spatial.distance import pdist, squareform
from scipy.spatial.distance import cdist
### Other Imports
import warnings
from copy import deepcopy
from dataclasses import dataclass, field
from typing import List, Dict, Tuple, Any
from abc import ABC, abstractmethod
from matplotlib.colors import LinearSegmentedColormap


#### Section 2: Algorithms

In [42]:
class Tour:
    def __init__(self, num_vehicles):
        self.routes = [[] for _ in range(num_vehicles)]
        self.ev_categories = ['' for _ in range(num_vehicles)]
        self.route_loads = [0 for _ in range(num_vehicles)]
        self.delivery_times = []

In [43]:
class RoutingAlgorithmBaseEV:
    def __init__(self):
        self.best_solution = None

    def Set_EV_Parameters(self):
        self.initial_charging = 100  # Start with full battery
        self.ev_speed = 25  # km/h (constant speed)
        
        # Define all EV categories and their specifications
        self.ev_categories = {
            'small': {
                'battery_capacity': 35,    # kWh
                'base_weight': 1500,       # kg
                'load_capacity': 500,      # kg
            },
            'medium': {
                'battery_capacity': 40,     # kWh
                'base_weight': 1800,        # kg
                'load_capacity': 600,       # kg
            },
            'large': {
                'battery_capacity': 45,     # kWh
                'base_weight': 2000,        # kg
                'load_capacity': 700,       # kg
            },
            'xlarge': {
                'battery_capacity': 50,     # kWh
                'base_weight': 2200,        # kg
                'load_capacity': 800,       # kg
            }
        }
        
        # Energy consumption parameters
        self.energy_consumption_rate = 0.15  # Base rate in kWh/km
        self.weight_factor = 0.05           # Additional consumption per 1000kg
        self.battery_safety_margin = 20     # Minimum battery level (%)
        
        print("\nEV Fleet Configuration:")
        print("Dynamic vehicle selection enabled")
        print("Flexible fleet size enabled")
        print("\nAvailable Vehicle Categories:")
        for category, specs in self.ev_categories.items():
            print(f"\n{category.upper()}:")
            print(f"Load Capacity: {specs['load_capacity']} kg")


    def Set_Charging_Station_Parameters(self, charging_rate, charging_cost):
        self.charging_rate = charging_rate
        self.charging_cost = charging_cost

    def Set_Tour_Parameters(self, distance_matrix, distance_between_charging_stations_customers, 
                          customer_locations, charging_station_locations):
        """Store tour parameters"""
        self.number_of_customers = len(distance_matrix) - 1  # Exclude depot
        self.number_of_charging_stations = len(distance_between_charging_stations_customers[0])
        self.distance_matrix = distance_matrix
        self.distance_between_charging_stations_customers = distance_between_charging_stations_customers
        self.customer_locations = customer_locations
        self.charging_station_locations = charging_station_locations
        self.depot_location = customer_locations[0]  # First location is depot

    def Set_Customer_Parameters(self, customer_items_weights, customer_deadlines):
        self.customer_items_weights = customer_items_weights
        self.customer_deadlines = customer_deadlines

    def Discharging_Rate_Per_KM(self, load):
        # Basic energy consumption model for EVs
        base_consumption = self.energy_consumption_rate  # Base consumption rate
        weight_factor = 0.05  # Additional consumption per kg of load
        total_weight = self.ev_base_weight + load
        
        power_consumption = base_consumption + (self.ev_base_weight+load) * weight_factor/1000
        # This Rate is in KWh/Km not percentage
        return (power_consumption / self.ev_battery_capacity) 

    def calculate_delivery_time(self, route):
        total_time = 0
        current_load = sum(self.customer_items_weights.get(str(customer - 1), 0) 
                        for customer in route if customer > 0)
        
        for i in range(len(route) - 1):
            current_location = route[i]
            next_location = route[i + 1]
            
            # Calculate travel time
            distance = self.calculate_distance(current_location, next_location)
            travel_time = distance / self.ev_speed
            total_time += travel_time
            
            # Add charging time if at charging station
            if current_location < 0:  # Charging station
                charging_time = self.ev_battery_capacity / self.charging_rate
                total_time += charging_time
                
            # Update load after delivery
            if current_location > 0:  # If current location is a customer
                current_load -= self.customer_items_weights.get(str(current_location - 1), 0)
        
        return total_time

    def calculate_fitness(self, tour):
        delivery_times = []
        
        # Calculate delivery time for each route
        for route in tour.routes:
            delivery_time = self.calculate_delivery_time(route)
            delivery_times.append(delivery_time)
            
        # Store individual delivery times for analysis
        tour.delivery_times = delivery_times
        
        # Fitness is the maximum delivery time (to be minimized)
        max_delivery_time = max(delivery_times) if delivery_times else float('inf')
        
        return max_delivery_time
    def calculate_travel_cost(self, route):
        total_distance = 0    
        for i in range(len(route) - 1):
            current_location = route[i]
            next_location = route[i + 1]
            
            distance = self.calculate_distance(current_location, next_location)
            total_distance += distance
            
        return total_distance

    def calculate_delay_cost(self, route):
        delay_cost = 0
        current_time = 0
        
        for i in range(1, len(route) - 1):
            current_location = route[i]
            if current_location > 0:  # Customer location
                deadline = self.customer_deadlines[str(current_location - 1)]
                if current_time > deadline:
                    delay_cost += current_time - deadline
            
            next_location = route[i + 1]
            if current_location < 0:  # Charging station
                charging_time = self.ev_battery_capacity / self.charging_rate
                current_time += charging_time
            else:
                distance = self.calculate_distance(current_location, next_location)
                current_time += distance / self.ev_speed
        
        return delay_cost * self.ev_speed

    def find_nearest_charging_station(self, current_location):
        nearest_charging_station = 0
        min_distance = float('inf')
        
        for charging_station in range(self.number_of_charging_stations):
            distance = self.distance_between_charging_stations_customers[current_location][charging_station]
            if distance < min_distance:
                min_distance = distance
                nearest_charging_station = charging_station
                
        return nearest_charging_station

    def calculate_power_needed(self, from_location, to_location, load):
        distance = self.calculate_distance(from_location, to_location)
        return self.Discharging_Rate_Per_KM(load) * distance

    def insert_charging_stations(self, route):
        new_route = [0]  # Start at depot
        current_battery = self.initial_charging
        battery_safety_margin = 20  # 20% safety margin
        
        # Calculate initial load
        current_load = sum(self.customer_items_weights.get(str(customer - 1), 0) 
                         for customer in route if customer > 0)
        
        for i in range(1, len(route)):
            current_location = route[i]
            power_needed = self.calculate_power_needed(new_route[-1], current_location, current_load)
            
            # Check if charging is needed
            if current_battery - power_needed < max(self.initial_charging * (battery_safety_margin / 100), 
                                                  power_needed * 1.5):
                if new_route[-1] >= 0:
                    nearest_charging_station = self.find_nearest_charging_station(new_route[-1])
                    if nearest_charging_station is None:
                        raise ValueError(f"No charging station found near location {new_route[-1]}")
                    new_route.append(-(nearest_charging_station + 1))
                    current_battery = self.initial_charging
                    power_needed = self.calculate_power_needed(-(nearest_charging_station + 1), 
                                                            current_location, current_load)
            
            new_route.append(current_location)
            current_battery -= power_needed
            
            # Update load after delivery
            if current_location > 0:
                current_load = max(0, current_load - self.customer_items_weights.get(str(current_location - 1), 0))
        
        # Check if final charging needed for return to depot
        final_power_needed = self.calculate_power_needed(new_route[-1], 0, current_load)
        if current_battery - final_power_needed < self.initial_charging * (battery_safety_margin / 100):
            nearest_charging_station = self.find_nearest_charging_station(new_route[-1])
            if nearest_charging_station is not None:
                new_route.append(-(nearest_charging_station + 1))
        
        return new_route

    def calculate_distance(self, from_location, to_location):
        if from_location >= 0 and to_location >= 0:
            return self.distance_matrix[from_location][to_location]
        elif from_location < 0 and to_location >= 0:
            charging_station_index = -from_location - 1
            return self.distance_between_charging_stations_customers[to_location][charging_station_index]
        elif from_location >= 0 and to_location < 0:
            charging_station_index = -to_location - 1
            return self.distance_between_charging_stations_customers[from_location][charging_station_index]
        else:
            print(f"Warning: Invalid route segment from {from_location} to {to_location}")
            return 0

In [46]:
class SimulatedAnnealingEVRP(RoutingAlgorithmBaseEV):
    def __init__(self, 
                 initial_temperature=1000.0,
                 cooling_rate=0.95,
                 min_temperature=1.0,
                 iterations_per_temp=100):
        # Call parent class constructor
        super().__init__()
        
        # SA specific parameters
        self.initial_temperature = initial_temperature
        self.cooling_rate = cooling_rate
        self.min_temperature = min_temperature
        self.iterations_per_temp = iterations_per_temp
        self.best_solution = None
        self.best_fitness = float('inf')
        self.best_fitness_history = []

    def initialize_solution(self, customer_locations, customer_demands):
        print(f"Initializing solution with {len(customer_locations)} customers")
        print(f"Customer demands: {customer_demands}")
        
        # Step 1: Calculate initial number of clusters based on total demand
        total_demand = sum(customer_demands)
        # Use smallest vehicle capacity as baseline to ensure feasibility
        num_clusters = math.ceil(total_demand / self.ev_categories['small']['load_capacity'])
        print(f"Number of clusters: {num_clusters}")
        
        # Step 2: Perform K-means clustering to group nearby customers
        kmeans = KMeans(n_clusters=num_clusters, random_state=42)
        cluster_assignments = kmeans.fit_predict(customer_locations)
        print(f"Cluster assignments: {cluster_assignments}")
        
        # Step 3: Create Tour object with initial empty routes
        solution = Tour(num_clusters)
        
        # Step 4: Build routes for each cluster
        for i in range(num_clusters):
            # Get customers in this cluster (add 1 to indices since we index from 1)
            cluster_customers = [j+1 for j, cluster in enumerate(cluster_assignments) 
                            if cluster == i]
            print(f"Cluster {i} customers: {cluster_customers}")
            
            if not cluster_customers:
                continue
            
            # Create route using nearest neighbor heuristic
            route = self._create_route_from_cluster(cluster_customers, customer_locations)
            
            # Calculate total load for this route
            route_load = sum(self.customer_items_weights[str(c-1)] for c in cluster_customers)
            
            # Step 5: Assign appropriate vehicle type based on load
            for vehicle_type, specs in sorted(self.ev_categories.items(), 
                                            key=lambda x: x[1]['load_capacity']):
                if specs['load_capacity'] >= route_load:
                    solution.ev_categories[i] = vehicle_type
                    break
            
            solution.routes[i] = route
            solution.route_loads[i] = route_load
        
        # Step 6: Clean up empty routes
        solution.routes = [r for r in solution.routes if r]
        solution.ev_categories = [ev for ev, r in zip(solution.ev_categories, solution.routes) if r]
        solution.route_loads = [rl for rl, r in zip(solution.route_loads, solution.routes) if r]
        
        return solution
    
    def _create_route_from_cluster(self, cluster_customers, customer_locations):
        print(f"Creating route for customers: {cluster_customers}")
        print(f"Customer locations length: {len(customer_locations)}")
        
        route = [0]  # Start at depot
        unvisited = cluster_customers.copy()
        
        while unvisited:
            current = route[-1]
            if current == 0:
                current_loc = self.depot_location
            else:
                # Since customer_locations doesn't include depot, adjust index
                current_loc = customer_locations[current - 1]  # Subtract 1 here
                
            # Find nearest unvisited customer
            min_dist = float('inf')
            nearest = None
            
            for customer in unvisited:
                # Adjust index for customer_locations
                customer_loc = customer_locations[customer - 1]  # Subtract 1 here
                dist = euclidean_distance(current_loc, customer_loc)
                if dist < min_dist:
                    min_dist = dist
                    nearest = customer
            
            route.append(nearest)
            unvisited.remove(nearest)
        
        route.append(0)  # Return to depot
        print(f"Created route: {route}")
        return route
    
    def _select_vehicle_type(self, load):
        """Select appropriate vehicle type based on load"""
        for vehicle_type, specs in sorted(self.ev_categories.items(), 
                                        key=lambda x: x[1]['load_capacity']):
            if specs['load_capacity'] >= load:
                return vehicle_type
        raise ValueError("No vehicle type can handle the load")
    
    def calculate_fitness(self, solution):
        """Calculate fitness (maximum route duration)"""
        max_duration = float('-inf')
        route_durations = []
        
        for route, vehicle_type in zip(solution.routes, solution.vehicle_types):
            duration = self._calculate_route_duration(route, vehicle_type)
            route_durations.append(duration)
            max_duration = max(max_duration, duration)
        
        solution.delivery_times = route_durations
        return max_duration
    
    def _calculate_route_duration(self, route, vehicle_type):
        """Calculate total duration of a route"""
        total_time = 0
        
        for i in range(len(route) - 1):
            from_node = route[i]
            to_node = route[i + 1]
            
            # Calculate travel time
            distance = self.calculate_distance(from_node, to_node)
            travel_time = distance / self.ev_speed
            total_time += travel_time
        
        return total_time
    
    def accept_solution(self, current_fitness, new_fitness, temperature):
        """Determine whether to accept new solution"""
        if new_fitness < current_fitness:
            return True
        
        # Calculate acceptance probability
        delta = new_fitness - current_fitness
        acceptance_probability = math.exp(-delta / temperature)
        
        return random.random() < acceptance_probability
    
    def optimize(self, customer_locations, customer_demands, max_iterations=1000):
        """Main simulated annealing algorithm"""
        # Initialize solution
        current_solution = self.initialize_solution(customer_locations, customer_demands)
        current_fitness = self.calculate_fitness(current_solution)
        
        self.best_solution = deepcopy(current_solution)
        self.best_fitness = current_fitness
        
        temperature = self.initial_temperature
        iteration = 0
        
        # Initialize neighborhood operations
        neighborhood_ops = NeighborhoodOperations()
        
        while temperature > self.min_temperature and iteration < max_iterations:
            for i in range(self.iterations_per_temp):
                # Select random neighborhood operation
                operation = random.choice([
                    'swap_between_routes',
                    'relocate_between_routes',
                    'two_opt_within_route',
                    'relocate_within_route'
                ])
                
                # Generate new solution using selected operation
                new_solution = self._apply_operation(current_solution, 
                                                  operation, 
                                                  neighborhood_ops)
                
                # Calculate fitness of new solution
                new_fitness = self.calculate_fitness(new_solution)
                
                # Decide whether to accept new solution
                if self.accept_solution(current_fitness, new_fitness, temperature):
                    current_solution = new_solution
                    current_fitness = new_fitness
                    
                    # Update best solution if improved
                    if new_fitness < self.best_fitness:
                        self.best_solution = deepcopy(new_solution)
                        self.best_fitness = new_fitness
                
                iteration += 1
                self.best_fitness_history.append(self.best_fitness)
            
            # Cool down
            temperature *= self.cooling_rate
            
            # Print progress
            if iteration % 100 == 0:
                print(f"Iteration {iteration}, Temperature: {temperature:.2f}, "
                      f"Best Fitness: {self.best_fitness:.2f}")
        
        return self.best_solution, self.best_fitness
    
    def _apply_operation(self, solution, operation, neighborhood_ops):
        """Apply selected neighborhood operation"""
        new_solution = deepcopy(solution)
        
        if operation == 'swap_between_routes':
            if len(solution.routes) < 2:
                return new_solution
                
            r1, r2 = random.sample(range(len(solution.routes)), 2)
            pos1 = random.randint(1, len(solution.routes[r1]) - 2)
            pos2 = random.randint(1, len(solution.routes[r2]) - 2)
            
            new_solution = neighborhood_ops.swap_between_routes(
                new_solution, r1, r2, pos1, pos2)
                
        elif operation == 'relocate_between_routes':
            if len(solution.routes) < 2:
                return new_solution
                
            source = random.randint(0, len(solution.routes) - 1)
            dest = random.randint(0, len(solution.routes) - 1)
            while dest == source:
                dest = random.randint(0, len(solution.routes) - 1)
                
            pos = random.randint(1, len(solution.routes[source]) - 2)
            
            new_solution = neighborhood_ops.relocate_between_routes(
                new_solution, source, dest, pos)
                
        elif operation == 'two_opt_within_route':
            route_idx = random.randint(0, len(solution.routes) - 1)
            route_len = len(solution.routes[route_idx])
            if route_len <= 4:  # Need at least 2 customers for 2-opt
                return new_solution
                
            i = random.randint(1, route_len - 3)
            j = random.randint(i + 1, route_len - 2)
            
            new_solution = neighborhood_ops.two_opt_within_route(
                new_solution, route_idx, i, j)
                
        elif operation == 'relocate_within_route':
            route_idx = random.randint(0, len(solution.routes) - 1)
            route_len = len(solution.routes[route_idx])
            if route_len <= 3:  # Need at least 2 customers for relocation
                return new_solution
                
            pos1 = random.randint(1, route_len - 2)
            pos2 = random.randint(1, route_len - 2)
            while pos2 == pos1:
                pos2 = random.randint(1, route_len - 2)
                
            new_solution = neighborhood_ops.relocate_within_route(
                new_solution, route_idx, pos1, pos2)
        
        return new_solution

In [47]:
def euclidean_distance(point1, point2):
    return np.sqrt(np.sum((np.array(point1) - np.array(point2)) ** 2))

def create_distance_matrix(locations):
    """Create distance matrix using Euclidean distance"""
    n = len(locations)
    matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            matrix[i][j] = euclidean_distance(locations[i], locations[j])
    return matrix

def create_charging_distance_matrix(locations, charging_stations):
    """Create distance matrix between locations and charging stations"""
    matrix = np.zeros((len(locations), len(charging_stations)))
    for i, loc in enumerate(locations):
        for j, station in enumerate(charging_stations):
            matrix[i][j] = euclidean_distance(loc, station)
    return matrix
def run_ev_routing_test(toml_file_path):
    """Execute EV routing test with dynamic fleet size and vehicle selection"""
    # Load and process TOML file
    print(f"Loading data from {toml_file_path}...")
    try:
        data = toml.load(toml_file_path)
        print("Data loaded successfully!")
    except Exception as e:
        print(f"Error loading TOML file: {e}")
        return

    # Create distance matrices using Euclidean distance
    customer_locations = [data['depot_location']] + data['customer_locations']  # Add depot
    charging_stations = data['charging_stations']
    
    print("\nProblem Size:")
    print(f"Number of Customers: {len(data['customer_locations'])}")
    print(f"Number of Charging Stations: {len(charging_stations)}")
    print(f"Total Delivery Weight: {sum(data['customer_items_weights'])} kg")
    
    # Create distance matrices
    distance_matrix = create_distance_matrix(customer_locations)
    distance_between_charging_stations_customers = create_charging_distance_matrix(
        customer_locations, charging_stations
    )
    
    # Initialize algorithm
    print("\nInitializing Simulated Annealing algorithm...")
    sa = SimulatedAnnealingEVRP(
        initial_temperature=1000,
        cooling_rate=0.95,
        min_temperature=1,
        iterations_per_temp=100
    )
    
    # Set base parameters
    print("\nSetting algorithm parameters...")
    sa.Set_EV_Parameters()
    
    sa.Set_Charging_Station_Parameters(
        charging_rate=data['charging_rate'],
        charging_cost=1.0
    )
    
    sa.Set_Tour_Parameters(
        distance_matrix=distance_matrix,
        distance_between_charging_stations_customers=distance_between_charging_stations_customers,
        customer_locations=customer_locations,  # This includes depot at index 0
        charging_station_locations=charging_stations
    )
    
    # Set customer parameters
    customer_weights = {str(i): w for i, w in enumerate(data['customer_items_weights'])}
    customer_deadlines = {str(i): float('inf') for i in range(len(data['customer_items_weights']))}
    
    sa.Set_Customer_Parameters(
        customer_items_weights=customer_weights,
        customer_deadlines=customer_deadlines
    )
    
    # Run optimization
    print("\nStarting optimization...")
    print("This may take a few minutes...")
    best_solution, best_fitness = sa.optimize(
        customer_locations[1:],  # Exclude depot from clustering
        data['customer_items_weights'],
        max_iterations=200
    )
    
    # Print results
    print("\n=== Optimization Results ===")
    print(f"Best fitness (maximum delivery time): {best_fitness:.2f} hours")
    print(f"Number of vehicles used: {len(best_solution.routes)}")
    
    print("\nDetailed route information:")
    total_distance = 0
    total_customers = 0
    total_energy = 0
    
    for i, route in enumerate(best_solution.routes):
        print(f"\nRoute {i+1}:")
        print(f"Vehicle Type: {best_solution.vehicle_types[i].upper()}")
        print(f"Load: {best_solution.route_loads[i]} kg")
        print("Sequence:", ' -> '.join(map(str, route)))
        print(f"Delivery time: {best_solution.delivery_times[i]:.2f} hours")
        
        # Calculate route statistics
        route_distance = sum(euclidean_distance(
            customer_locations[route[j]] if route[j] >= 0 else charging_stations[-route[j]-1],
            customer_locations[route[j+1]] if route[j+1] >= 0 else charging_stations[-route[j+1]-1]
        ) for j in range(len(route)-1))
        
        # Get vehicle specs for this route
        vehicle_specs = sa.ev_categories[best_solution.vehicle_types[i]]
        
        # Calculate energy consumption
        base_consumption = 0.15  # kWh/km
        weight_factor = 0.05  # Additional consumption per 1000kg
        total_weight = vehicle_specs['base_weight'] + best_solution.route_loads[i]
        energy_consumption = (base_consumption + (total_weight * weight_factor/1000)) * route_distance
        
        num_customers = sum(1 for loc in route if loc > 0)
        num_charges = sum(1 for loc in route if loc < 0)
        
        print(f"Distance traveled: {route_distance:.2f} km")
        print(f"Energy consumed: {energy_consumption:.2f} kWh")
        print(f"Customers served: {num_customers}")
        print(f"Charging stops: {num_charges}")
        
        total_distance += route_distance
        total_customers += num_customers
        total_energy += energy_consumption
    
    print("\n=== Overall Statistics ===")
    print(f"Total vehicles used: {len(best_solution.routes)}")
    print(f"Vehicle type distribution:")
    vehicle_counts = {}
    for vehicle_type in best_solution.vehicle_types:
        vehicle_counts[vehicle_type] = vehicle_counts.get(vehicle_type, 0) + 1
    for category, count in vehicle_counts.items():
        print(f"  {category.upper()}: {count}")
    
    print(f"\nTotal distance: {total_distance:.2f} km")
    print(f"Total energy consumption: {total_energy:.2f} kWh")
    print(f"Average energy per km: {total_energy/total_distance:.3f} kWh/km")
    print(f"Total customers served: {total_customers}")
    print(f"Average distance per customer: {total_distance/total_customers:.2f} km")
    
    # Plot fitness history
    plt.figure(figsize=(12, 6))
    plt.plot(sa.best_fitness_history, 'b-', label='Best Fitness')
    plt.title('Optimization Progress')
    plt.xlabel('Iteration')
    plt.ylabel('Best Fitness (Maximum Delivery Time)')
    plt.grid(True)
    plt.legend()
    plt.show()
    
    # Plot route visualization
    plt.figure(figsize=(15, 15))
    
    # Plot depot
    plt.plot(customer_locations[0][0], customer_locations[0][1], 'k^', 
             markersize=15, label='Depot')
    
    # Plot charging stations
    charging_stations = np.array(charging_stations)
    plt.plot(charging_stations[:, 0], charging_stations[:, 1], 'gs', 
             markersize=10, label='Charging Stations')
    
    # Plot customers
    customers = np.array(customer_locations[1:])
    plt.plot(customers[:, 0], customers[:, 1], 'bo', 
             markersize=8, label='Customers')
    
    # Plot routes with different colors
    colors = plt.cm.rainbow(np.linspace(0, 1, len(best_solution.routes)))
    for route, color in zip(best_solution.routes, colors):
        route_coords = []
        for loc in route:
            if loc >= 0:
                route_coords.append(customer_locations[loc])
            else:
                route_coords.append(charging_stations[-loc-1])
        route_coords = np.array(route_coords)
        plt.plot(route_coords[:, 0], route_coords[:, 1], '-', 
                 color=color, alpha=0.7, linewidth=2)
    
    plt.title('Optimized Routes')
    plt.xlabel('X Coordinate')
    plt.ylabel('Y Coordinate')
    plt.grid(True)
    plt.legend()
    plt.show()
    
    return sa, best_solution, best_fitness

if __name__ == "__main__":
    # Required imports
    import numpy as np
    import toml
    import matplotlib.pyplot as plt
    from sklearn.cluster import KMeans
    import math
    from copy import deepcopy
    import random
    
    # File path
    toml_path = "/Users/chanakyavasantha/Comsets/test_cases/customers_10/c10_1.toml"
    

    sa, solution, fitness = run_ev_routing_test(toml_path)
    print("\nTest completed successfully!")

Loading data from /Users/chanakyavasantha/Comsets/test_cases/customers_10/c10_1.toml...
Data loaded successfully!

Problem Size:
Number of Customers: 10
Number of Charging Stations: 10
Total Delivery Weight: 710 kg

Initializing Simulated Annealing algorithm...

Setting algorithm parameters...

EV Fleet Configuration:
Dynamic vehicle selection enabled
Flexible fleet size enabled

Available Vehicle Categories:

SMALL:
Load Capacity: 500 kg

MEDIUM:
Load Capacity: 600 kg

LARGE:
Load Capacity: 700 kg

XLARGE:
Load Capacity: 800 kg

Starting optimization...
This may take a few minutes...
Initializing solution with 10 customers
Customer demands: [50, 55, 90, 55, 80, 70, 85, 95, 80, 50]
Number of clusters: 2
Cluster assignments: [1 1 1 0 0 0 1 1 0 1]
Cluster 0 customers: [4, 5, 6, 9]
Creating route for customers: [4, 5, 6, 9]
Customer locations length: 10
Created route: [0, 4, 9, 5, 6, 0]
Cluster 1 customers: [1, 2, 3, 7, 8, 10]
Creating route for customers: [1, 2, 3, 7, 8, 10]
Customer loc

AttributeError: 'Tour' object has no attribute 'vehicle_types'